## It is often the case that we only want to use part of a input sequence. In the case of a sentence, we would like to "cover" later parts of the sentence for next token prediction, otherwise it will be trivial for the model to grab the prediction token from the input sentence. TODO: expand this on training algorithm that necessitates this, and why use -inf to achieve this, cool comparison trick to make masks; For now straight to the point -- train masked model

In [2]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext
from torchtext.data.utils import get_tokenizer
from torch.nn import TransformerEncoder, TransformerEncoderLayer


In [3]:
# what are we having from the data??

# first let's do data wrangling

TEXT = torchtext.data.Field(tokenize=get_tokenizer("basic_english"),
                            init_token='<sos>',
                            eos_token='<eos>',
                            lower=True)
train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)
TEXT.build_vocab(train_txt)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cpu'
def batchify(data, bsz):
    data = TEXT.numericalize([data.examples[0].text])
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

batch_size = 3
eval_batch_size = 10
train_data = batchify(train_txt, batch_size)
val_data = batchify(val_txt, eval_batch_size)
test_data = batchify(test_txt, eval_batch_size)

# params and util fn for training
bptt = 5
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [4]:
data, targets = get_batch(train_data, 0)

In [6]:
data # bsz 3, bptt 5

tensor([[   3,  158,   27],
        [  12,    9,  123],
        [3852,  296, 4173],
        [3872, 8105,  243],
        [ 884,    4,    6]])

In [8]:
targets.size()

torch.Size([15])